# Thông tin nhóm 14

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# 1. Câu hỏi

Cho các thông tin về căn nhà:

- Giá cho thuê trước khi bán.
- Địa chỉ.
- Loại nhà.
- Số phòng ngủ.
- Số nhà vệ sinh.
- Diện tích phần nhà.
- Diện tích toàn bộ lô đất.
- Năm xây dựng.
- Tiền thuế.
- Gần mấy trường học.
- Tình hình tội phạm.
- ...

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# 2. Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

# 3. Thu thập dữ liệu

## 3.1. Parse HTML

Dữ liệu thu thập từ trang web https://www.realtytrac.com/. Ta chỉ thu thập dữ liệu "các estate đã được bán tại  California", ta có HTML cần parse: https://m.realtytrac.com/mapsearch/sold/ca/

Thời gian lấy dữ liệu: Ngày 10/12/2019

In [ ]:
# Import các thư viện cần thiết.
import urllib.robotparser
import json
import os.path
import re

from requests_html import HTMLSession

In [ ]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://m.realtytrac.com/robots.txt')
rp.read()

In [ ]:
# Lấy link chứa danh sách các ngôi nhà đã bán theo Quận tại California
check_parse = rp.can_fetch('*', 'https://m.realtytrac.com/mapsearch/sold/ca/')

base_url = "https://m.realtytrac.com"
list_url_county = []

# Kiểm tra việc parse HTML được cho phép hay không?
if check_parse == True:
    session = HTMLSession()
    r = session.get('https://m.realtytrac.com/mapsearch/sold/ca/')

    counties = r.html.find("option")

    for county in counties:
        tail_url = county.attrs['value']
        if (tail_url):
            url = base_url + tail_url
            list_url_county.append(url)


list_url_county

In [ ]:
# Hàm trả về generator, generator này trả về trang tiếp theo sau mỗi lần duyệt.
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p-{i}"
        i += 1


# Hàm parseDetailPage: parse để lấy thông tin chi tiết của từng căn nhà từ detailUrl (của căn nhà đã bán).
# Tham số: session và detailUrl của căn nhà cần parse.
# Trả về: dictionary chứa các thông tin đã parse được.
def parseDetailPage(session: HTMLSession, detailUrl: str):

    # Lấy mã HTML của trang web.
    r = session.get(detailUrl)

    # Dictionary lưu lại kết quả.
    result = {}

    # Lấy address của estate.
    address = {}
    street = r.html.find("[itemprop = 'streetAddress']", first=True)
    if street is not None:
        street = street.text

    locality = r.html.find("[itemprop = 'addressLocality']", first=True)
    if locality is not None:
        locality = locality.text

    region = r.html.find("[itemprop = 'addressRegion']", first=True)
    if region is not None:
        region = region.text

    code = r.html.find("[itemprop = 'postalCode']", first=True)
    if code is not None:
        code = code.text

    address["street"] = street
    address["locality"] = locality
    address["region"] = region
    address["code"] = code

    result["address"] = address

    # Lấy thông tin ngày đã bán căn nhà.
    date_sold = r.html.find(".recently-sold", first=True)
    if date_sold is not None:
        result["date_sold"] = date_sold.text.replace("SOLD ON ", "")
    else:
        result["date_sold"] = None

    # Lấy giá cho thuê trước khi bán.
    mortgage = r.html.find(".property-price-info", first=True)
    if mortgage is not None:
        result["mortgage"] = mortgage.text.replace("Est. Mortgage: ","")
    else:
        result["mortgage"] = None


    # Lấy phân mô tả chi tiết về căn nhà.
    details = []
    detailTexts = r.html.find(".detail-text")
    if detailTexts is not None:
        for detailText in detailTexts:
            details.append(detailText.html)  # sở dĩ lấy mà HTML để dễ parse sau này (khi muốn lấy thông tin từ details)
    result["details"] = details

    # Lấy property info của căn nhà.
    propertyInfo = r.html.find(".property-info", first=True)
    if propertyInfo is not None:
        info = {}
        items = propertyInfo.find(".item")
        if items is not None:
            for item in items:
                name = item.find(".name", first=True).text.lower().replace(" ", "_")
                value = item.find(".value", first=True).text
                info[name] = value

        result["info"] = info
    else:
        result["info"] = None

    # Lấy property taxes của căn nhà.
    property_taxes = r.html.find(".property-taxes")
    if property_taxes is not None:
        for tax in property_taxes:
            key = tax.find(".section-head", first=True).text.lower().replace(" ", "_")
            taxes = {}
            items = tax.find(".item")
            if items is not None:
                for item in items:
                    name = item.find(".name", first=True).text.lower().replace(" ", "_")
                    value = item.find(".value", first=True).text
                    taxes[name] = value

            result[key] = taxes

    # Lấy số lượng trường học gần đó.
    local_school = r.html.find(".property-schools", first=True)
    num_school = 0
    if local_school is not None:
        num_school = len(local_school.find(".schoolInfo"))

    result["school"] = num_school

    # Lấy thông tin tội phạm trong vùng.
    crimes = {}
    local_crime_index = r.html.find(".property-local-crime", first=True)
    if local_crime_index is not None:
        type_crimes = local_crime_index.find(".gradeTitle")
        if type_crimes is not None:
            for type_crime in type_crimes:
                match = re.match(r"(.+?) = (\d+)", type_crime.text)
                if match is not None:
                    key = match.group(1).lower().replace(' ', '_')
                    value = int(match.group(2))
                    crimes[key] = value

    result["local_crime_index"] = crimes

    # Lấy số lượng nhà bị tịch thu gần đó.
    nearby_foreclosures = r.html.find(".property-nearby-fc", first=True)
    num_fc = 0
    if nearby_foreclosures is not None:
        num_fc = len(nearby_foreclosures.find(".nearby-property"))

    result["foreclosures"] = num_fc

    return result

In [ ]:
# Hàm parseSearchPage: parse theo page (mỗi parse chứa danh sách và url của 25 căn nhà đã được bán).
# Tham số: session và url của page hiện tại.
# Trả về: list thông tin chi tiết của tất cả căn nhà tại page hiện tại và url của page kế tiếp.
def parseSearchPage(session: HTMLSession, urlPage: str):

    # Lấy mã HTML.
    r = session.get(urlPage)

    # Lấu detailUrl của tất cả bất động sản.
    detailUrls = [link for link in r.html.absolute_links if 'property' in link]

    results = []  # List chứa kết quả parse được tại page.

    # Tiến hành parse từ detailUrl
    for detailUrl in detailUrls:
        # Kiểm tra việc parse HTML được cho phép hay không?
        if not rp.can_fetch('*', detailUrl):
            print(f"SKIP: {detailUrl}")
            continue

        print(f"Parsing detail url {detailUrl}")
        result = parseDetailPage(session, detailUrl)
        results.append(result)

    nextPageAnchor = r.html.find(".current + .page", first=True)
    return results, nextPageAnchor is not None

In [ ]:
# Hàm getAllCountyProperties: parse tất cả các page của một County để lấy thông tin chi tiết của 
# căn nhà được bán thành công.
# Tham số: session và countryUrl chứa thông tin các căn nhà được bán của một County.
# Trả về: File chứa kết quả đã parse thành công (tên file là tên County được parse).
def getAllCountyProperties(session: HTMLSession, countyUrl: str):
    # Lấy tên County từ urlCounty.
    if countyUrl.endswith("/"):
        countyUrl = countyUrl[:-1]
    fileName = f"data/{os.path.basename(countyUrl)}.json"

    # Kiểm tra sự tồn tại file.
    if os.path.exists(fileName):
        print(f"FILE EXISTS: {fileName}")
        return

    # Mở file để ghi dữ liệu (kiểu file là json)
    with open(fileName, "a") as fOut:
        for url in urlGenerator(countyUrl, 1):
            # Kiểm tra việc parse HTML được cho phép hay không?
            if not rp.can_fetch('*', url):
                print(f"SKIP: {url}")
                continue

            # Tiến hành parse theo từng page.
            print(f"Parsing {url}")
            results, nextPage = parseSearchPage(session, url)

            # Ghi kết quả xuống file.
            for result in results:
                json.dump(result, fOut, ensure_ascii=False)
                fOut.write("\n")

            # Kiểm tra có còn page tiếp theo hay không?
            if not nextPage:
                break

            # Mỗi lần parse 1 page thì cho sleep 2s.
            time.sleep(2)

Tiến hành lấy dữ liệu. Lưu ý trong quá trình lấy có thể bị timeout do đường truyền mạng, nếu bị thì chỉ cần làm theo các bước sau:
- Xem đã lấy tới page mấy. vd: "...page-125"
- Đổi lại tên file trong thư mục data là "orange-county-1.json" chẳng hạn (số 1, 2, 3,.. là theo mỗi lần lấy dữ liệu của mình).
- Vào hàm "getAllCountyProperties" đổi lại dòng "for url in urlGenerator(countyUrl, 1)" thành "for url in urlGenerator(countyUrl, 125)", số 125 là tùy mình lấy tới page thứ mấy mà bị timeout.
- Dữ liệu tất nhiên sẽ bi trùng, cứ việc yên tâm vì phần sau sẽ xóa trung.

In [ ]:
url = "https://m.realtytrac.com/mapsearch/sold/ca/orange-county"
session = HTMLSession()
getAllCountyProperties(session, url)

In [ ]:
# Phần gộp từng phần dữ liệu đã lấy dang dở lại thành 1 file.
# Nếu không bị timeout như trường hợp ở trên thì không dùng shell code này.
inputFiles = ["data/orange-county-1.json", "data/orange-county-2.json", 
              "data/orange-county-3.json", "data/orange-county-4.json"]
outputFile = "data/full-orange-county.json"

with open(outputFile, "a+") as fOut:
    for inputFile in inputFiles:
        with open(inputFile, "r") as fIn:
            data = fIn.read()
            fOut.write(data)

In [ ]:
# Phần convert file json sang file csv.

def xstr(s):
    return '' if s is None else str(s)


# Hàm preprocessFile: chuyển file json sang file csv.
# Tham số: tên file json và tên file csv.
# Không trả về.
def preprocessFile(inputFile: str, outputFile: str):

    # Kiểm tra tồn tại file Input.
    if os.path.exists(outputFile):
        print(f"FILE EXISTS: {outputFile}")
        return

    # Kiểm tra tồn tại file Output.
    if not os.path.exists(inputFile):
        print(f"Input file not found: {inputFile}")
        return

    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        tab = "\t"
        headers = (
            "address_street",
            "address_locality",
            "address_region",
            "address_code",
            "date_sold",
            "mortgage",
            "info_type",
            "info_bedrooms",
            "info_bathrooms",
            "info_size",
            "info_lot_size",
            "info_year_built",
            "info_est_value",
            "info_sold_price",
            "info_property_id",
            "info_county",
            "info_parcel_number",
            "taxes_land",
            "taxes_improvements",
            "taxes_total",
            "taxes_taxes",
            "school",
            "total_crime",
            "violent_crime",
            "property_crime",
            "foreclosures",
        )

        # Ghi các tên cột vào file output, mỗi cột ngăn cách bởi "\t".
        fOut.write(f"{tab.join(headers)}\n")

        # Duyệt từng line trong file Input (json).
        for line in fIn:
            row = json.loads(line)

            # Lấy địa chỉ
            address = row.get("address")
            address_street = address.get("street")
            address_locality = address.get("locality")
            address_region = address.get("region")
            address_code = address.get("code")

            # Lấy ngày bán thành công.
            date_sold = row.get("date_sold")

            # Lấy giá cho thuê mỗi tháng (trước khi được bán).
            mortgage = row.get("mortgage")
            if mortgage is not None:
                mortgage = float(mortgage.replace("$","").replace(",","").replace("/mo","").replace("Est. Refinance: ", ""))

            # Lấy phần info
            info = row.get("info")

            # Type.
            info_type = info.get("type")

            # Bedrooms.
            info_bedrooms = info.get("bedrooms")
            if info_bedrooms == "Contact Agent":
                info_bedrooms = None
            else:
                info_bedrooms = float(info_bedrooms)

            # Bathrooms.
            info_bathrooms = info.get("bathrooms")
            if info_bathrooms == "Contact Agent":
                info_bathrooms = None
            else:
                info_bathrooms = float(info_bathrooms)

            # Size.
            info_size = info.get("size")
            if info_size == "Contact Agent":
                info_size = None
            else:
                info_size = float(info_size.replace(",", "").replace(" sqft", ""))

            # Lot size.
            info_lot_size = info.get("lot_size")
            if info_lot_size == "Contact Agent":
                info_lot_size = None
            else:
                info_lot_size = float(info_lot_size.replace(",", "").replace(" sqft", "").replace(" acres", ""))

            # Year build.
            info_year_built = info.get("year_built")
            if info_year_built  == "Contact Agent":
                info_year_built = None
            else:
                info_year_built = int(info_year_built)

            # Est value.
            info_est_value = info.get("est._value")
            if info_est_value is not None:
                info_est_value = float(info_est_value.replace("$", "").replace(",",""))

            # Sold price.
            info_sold_price = info.get("sold_price")
            if info_sold_price is not None:
                if info_sold_price != "N/A":
                    info_sold_price = float(info_sold_price.replace("$", "").replace(",", ""))

            # ID.
            info_property_id = info.get("property_id")

            # County.
            info_county = info.get("county")

            # Parcel_number.
            info_parcel_number = info.get("parcel_number")

            # Lấy phần Taxes.
            taxes = row.get("property_taxes")
            taxes_land = None
            taxes_improvements = None
            taxes_total = None
            taxes_taxes = None
            if taxes is not None:
                taxes_land = float(taxes.get("land").replace("$", "").replace(",", ""))
                taxes_improvements = float(taxes.get("improvements").replace("$", "").replace(",", ""))
                taxes_total = float(taxes.get("total").replace("$", "").replace(",", ""))
                taxes_taxes = taxes.get("taxes").replace("$", "").replace(",", "")

            # Lấy số lượng scholl gần đó.
            school = int(row.get("school"))

            # Lấy tình hình tội phạm.
            crime = row.get("local_crime_index")
            if crime is not None:
                total_crime = crime.get("total_crime")
                if total_crime is not None:
                    total_crime = int(total_crime)
                violent_crime = crime.get("violent_crime")
                if violent_crime is not None:
                    violent_crime = int(violent_crime)
                property_crime = crime.get("property_crime")
                if property_crime is not None:
                    property_crime = int(property_crime)

            # Lấy số lượng foreclosures gần đó.
            foreclosures = int(row.get("foreclosures"))

            # Row này là tổng hợp các thông tin lấy được ở trên.
            row = (
                xstr(address_street),
                xstr(address_locality),
                xstr(address_region),
                xstr(address_code),
                xstr(date_sold),
                xstr(mortgage),
                xstr(info_type),
                xstr(info_bedrooms),
                xstr(info_bathrooms),
                xstr(info_size),
                xstr(info_lot_size),
                xstr(info_year_built),
                xstr(info_est_value),
                xstr(info_sold_price),
                xstr(info_property_id),
                xstr(info_county),
                xstr(info_parcel_number),
                xstr(taxes_land),
                xstr(taxes_improvements),
                xstr(taxes_total),
                xstr(taxes_taxes),
                xstr(school),
                xstr(total_crime),
                xstr(violent_crime),
                xstr(property_crime),
                xstr(foreclosures),
            )

            # Ghi xuống file output
            fOut.write(f"{tab.join(row)}\n")

&#9889; Sở dĩ nhóm không thêm phần `description` vào dữ liệu là bởi vì khi quan sát phần mã `html` của `description` thì phát hiện ra nó được viết theo một `format` nhất định, các thông tin số liệu của nó được lấy từ các bảng số liệu trong cùng một trang web (tức là các phần thông tin nhà, thuế,.. mà mình đã parse ở trên). Nếu số liệu nào có thì sẽ được lấy ra và tạo một câu mô tả theo `format` cho trước, còn số liệu nào không có thì hiển nhiên không xuất hiện câu đó. Như vậy, có thể kết luận là phần `description` lúc này sẽ không giúp ích gì cho chúng ta.

In [ ]:
inputFile = "data/full-orange-county.json"
outputFile = "data/orange-county.csv"
preprocessFile(inputFile, outputFile)

## 3.2. Lựa chọn dữ liệu của năm 2019 và có "correct ouput"

In [1]:
# Import các thư viện cần thiết.
import pandas as pd

### 3.2.1 Xóa dữ liệu trùng

#### Dữ liệu ban đầu

In [2]:
# Đọc dữ liệu lên
file_csv = "data/Orange-County/orange-county.csv"
data_df =  pd.read_csv(file_csv, sep='\t')
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21268 entries, 0 to 21267
Data columns (total 26 columns):
address_street        21204 non-null object
address_locality      21266 non-null object
address_region        21268 non-null object
address_code          21268 non-null int64
date_sold             21268 non-null object
mortgage              21134 non-null float64
info_type             21268 non-null object
info_bedrooms         19148 non-null float64
info_bathrooms        19159 non-null float64
info_size             20282 non-null float64
info_lot_size         14829 non-null float64
info_year_built       19737 non-null float64
info_est_value        1297 non-null float64
info_sold_price       19837 non-null float64
info_property_id      21268 non-null int64
info_county           21268 non-null object
info_parcel_number    21268 non-null int64
taxes_land            20981 non-null float64
taxes_improvements    20981 non-null float64
taxes_total           20981 non-null float64
taxe

Dữ liệu ban đầu có 21268 dòng.

#### Dữ liệu sau khi xóa trùng

In [4]:
# Xóa các phần dữ liệu bị trùng
unique_data_df = data_df.drop_duplicates()

In [5]:
# Xem phần dữ liệu khi đã xóa trùng
unique_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18893 entries, 0 to 21267
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

Dữ liệu sau khi xóa trùng còn lại 18893 dòng.

In [6]:
# Ghi dữ liệu xuống file csv
unique_data_df.to_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t', index=False)

### 3.2.2. Lấy dữ liệu của năm 2019

In [7]:
# Đọc dữ liệu lên
df = pd.read_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t')
df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18893 entries, 0 to 18892
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

In [9]:
# Thêm cột năm bán vào dữ liệu
df = df.assign(year_sold=df.date_sold.str[-4:])

In [10]:
# Xem dữ liệu này thu thập của các năm nào
df.year_sold.unique()

array(['2019', '1993', '1965', '1975', '1995', '1979', '1988', '2001',
       '1991', 'SOLD', '2017', '1973', '2005', '1981', '1969', '1989',
       '1976', '1977', '2002', '1984', '1987', '1994', '2003', '1968',
       '2015', '2012', '1971', '2006', '1998', '1967', '2011', '1986',
       '1992', '2018', '1970', '1999', '2000', '1972', '1985', '1974',
       '1978', '1980', '2013', '1996', '2008', '1983', '2007', '2014',
       '2016', '1982', '1997', '1990', '2004', '2010', '1964', '1966',
       '2009', '1963'], dtype=object)

In [11]:
# Chỉ lấy phần dữ liệu của năm 2019
df = df[df.year_sold == "2019"]
df.year_sold = df.year_sold.astype(int)

In [12]:
# Ghi dữ liệu xuống file csv
df.to_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t', index=False)

### 3.2.3. Lấy dữ liệu của năm 2019 có "correct output"

In [13]:
import pandas as pd

In [14]:
# Đọc dữ liệu
data_df = pd.read_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t')
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4,2019
1,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4,2019
2,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4,2019
3,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4,2019
4,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4,2019
5,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4,2019
6,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4,2019
7,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4,2019
8,13887 La Jolla Plz,Garden Grove,CA,92844,11/22/2019,2226.0,Single Family Residence,3.0,3.0,1637.0,...,332109.0,115034.0,447143.0,5673 (1.26 %),3,76.0,57.0,122.0,4,2019
9,163 Jaripol Cir,Rancho Mission Viejo,CA,92694,11/22/2019,1881.0,Condominium,1.0,2.0,921.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,2019


In [15]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16011 entries, 0 to 16010
Data columns (total 27 columns):
address_street        15963 non-null object
address_locality      16009 non-null object
address_region        16011 non-null object
address_code          16011 non-null int64
date_sold             16011 non-null object
mortgage              15976 non-null float64
info_type             16011 non-null object
info_bedrooms         14434 non-null float64
info_bathrooms        14444 non-null float64
info_size             15204 non-null float64
info_lot_size         10814 non-null float64
info_year_built       14734 non-null float64
info_est_value        221 non-null float64
info_sold_price       15755 non-null float64
info_property_id      16011 non-null int64
info_county           16011 non-null object
info_parcel_number    16011 non-null int64
taxes_land            15753 non-null float64
taxes_improvements    15753 non-null float64
taxes_total           15753 non-null float64
taxes

Dữ liệu có 16011 căn nhà được bán trong 2019

Ta chỉ lấy dữ liệu có `info_sold_price` khác `null`. Như đã nói ở phần cuối của `1.1. Parse HTML` thì không thể tìm kiếm các giá trị chính xác từ phần `decription` để có thể lắp đầy giá trị `null`, mà nếu có tìm cũng chỉ toàn ra giá trị trung bình của các căn nhà được bán lân cận chứ không phải giá trị chính xác của nó.

In [16]:
df = data_df[pd.notna(data_df["info_sold_price"])]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15755 entries, 0 to 16010
Data columns (total 27 columns):
address_street        15709 non-null object
address_locality      15753 non-null object
address_region        15755 non-null object
address_code          15755 non-null int64
date_sold             15755 non-null object
mortgage              15755 non-null float64
info_type             15755 non-null object
info_bedrooms         14248 non-null float64
info_bathrooms        14258 non-null float64
info_size             14975 non-null float64
info_lot_size         10639 non-null float64
info_year_built       14505 non-null float64
info_est_value        0 non-null float64
info_sold_price       15755 non-null float64
info_property_id      15755 non-null int64
info_county           15755 non-null object
info_parcel_number    15755 non-null int64
taxes_land            15502 non-null float64
taxes_improvements    15502 non-null float64
taxes_total           15502 non-null float64
taxes_t

Dữ liệu còn lại là 15755.

&#9889; Sau khi xóa các dòng có sold_price là `null`. Xem lại cột `info_est_value` thấy toàn `null`, cột này ý nghĩa là giá trị định giá của căn nhà trên các bản rao bán (chưa chốt giá). Nên xóa đi cột này là hoàn toàn hợp lý.

In [18]:
df = df.drop(columns=["info_est_value"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15755 entries, 0 to 16010
Data columns (total 26 columns):
address_street        15709 non-null object
address_locality      15753 non-null object
address_region        15755 non-null object
address_code          15755 non-null int64
date_sold             15755 non-null object
mortgage              15755 non-null float64
info_type             15755 non-null object
info_bedrooms         14248 non-null float64
info_bathrooms        14258 non-null float64
info_size             14975 non-null float64
info_lot_size         10639 non-null float64
info_year_built       14505 non-null float64
info_sold_price       15755 non-null float64
info_property_id      15755 non-null int64
info_county           15755 non-null object
info_parcel_number    15755 non-null int64
taxes_land            15502 non-null float64
taxes_improvements    15502 non-null float64
taxes_total           15502 non-null float64
taxes_taxes           15502 non-null object
scho

Ta cần kiểm tra xem ứng với `15755` dòng dữ liệu thì phải có đúng `15755` `info_property_id` hay không?

In [19]:
len(df.info_property_id.unique())

15755

OK ổn vì có đúng `15755` `info_property_id` như mong muốn

Bây giờ, ta kiểm tra liệu ứng với mỗi `address_street` liệu chỉ có duy nhất một căn nhà được bán thành công hay không?

In [20]:
address_street_count = df.address_street.value_counts()
address_street_count

1600 S Disneyland Dr          14
200 W Midway Dr                7
27 Arriate St                  6
14 Arriate St                  3
Situs Pending                  3
17331 Beach Blvd               3
14051 Newport Ave              3
2244 Via Puerta                3
135 Alienta Ln                 3
801 W Civic Center Dr          3
148 Alienta Ln                 3
24055 Paseo Del Lago # 1       3
212 Gaspar                     2
8320 E Meadowgate Dr           2
16715 Bayview Dr               2
23 Arriate St                  2
946 Sunlit Ln                  2
367 Hamilton St                2
8711 Masters Dr                2
26216 Sanz                     2
8177 Garfield Ave              2
2223 N Broadway                2
9300 Toledo Way                2
24 Granja Ct                   2
13882 Anita Pl                 2
16790 S Harbor Blvd            2
21991 El Toro Rd               2
2515 McCabe Way                2
20510 Regal Oaks Dr            2
22800 Savi Ranch Pkwy          2
          

Như vậy có `38` `address_street` có số lượng nhà bán thành công trong năm 2019 lơn hơn 1, trong đó lại có `1600 S Disneyland Dr` có số lần xuất hiện lên đến con sô `16`, điều này hơi kỳ lạ. Vì vậy, ta cần quan sát thêm về các `addresss_street` kiểu này.

In [21]:
# Lấy tên các addresss_street có số lượng nhà được bán trong năm 2019 nhiều hơn 1
duplicate_addresss_street = address_street_count[address_street_count > 1].index
duplicate_addresss_street

Index(['1600 S Disneyland Dr', '200 W Midway Dr', '27 Arriate St',
       '14 Arriate St', 'Situs Pending', '17331 Beach Blvd',
       '14051 Newport Ave', '2244 Via Puerta', '135 Alienta Ln',
       '801 W Civic Center Dr', '148 Alienta Ln', '24055 Paseo Del Lago # 1',
       '212 Gaspar', '8320 E Meadowgate Dr', '16715 Bayview Dr',
       '23 Arriate St', '946 Sunlit Ln', '367 Hamilton St', '8711 Masters Dr',
       '26216 Sanz', '8177 Garfield Ave', '2223 N Broadway', '9300 Toledo Way',
       '24 Granja Ct', '13882 Anita Pl', '16790 S Harbor Blvd',
       '21991 El Toro Rd', '2515 McCabe Way', '20510 Regal Oaks Dr',
       '22800 Savi Ranch Pkwy', '9666 Trask Ave', '2054 Orange Ave',
       '1540 E Heritage Pl', '2285 Via Puerta', '100 S Anaheim Blvd',
       '152 Alienta Ln', '34 Maple Leaf', '24055 Paseo Del Lago'],
      dtype='object')

In [22]:
len(duplicate_addresss_street)

38

Xem thử tất cả các dòng có `address_street` là `1600 S Disneyland Dr` (hay `duplicate_addresss_street[0]`)

In [23]:
tmp_df = df[df.address_street == duplicate_addresss_street[0]]
tmp_df

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
1360,1600 S Disneyland Dr,Anaheim,CA,92802,11/07/2019,27.0,Condominium,NaN,NaN,NaN,...,568692.0,3222588.0,3791280.0,42993 (1.13 %),3,88.0,71.0,127.0,4,2019
1896,1600 S Disneyland Dr,Anaheim,CA,92802,11/01/2019,91.0,Condominium,NaN,NaN,NaN,...,616940.0,3495992.0,4112932.0,46611 (1.13 %),3,88.0,71.0,127.0,4,2019
2520,1600 S Disneyland Dr,Anaheim,CA,92802,10/28/2019,161.0,Condominium,NaN,NaN,NaN,...,562456.0,3187251.0,3749707.0,42526 (1.13 %),3,88.0,71.0,127.0,4,2019
3986,1600 S Disneyland Dr,Anaheim,CA,92802,09/20/2019,132.0,Condominium,NaN,NaN,NaN,...,558705.0,3165994.0,3724699.0,42244 (1.13 %),3,88.0,71.0,127.0,4,2019
4172,1600 S Disneyland Dr,Anaheim,CA,92802,06/04/2019,139.0,Condominium,NaN,NaN,NaN,...,281385.0,1594512.0,1875897.0,21449 (1.14 %),3,88.0,71.0,127.0,4,2019
5068,1600 S Disneyland Dr,Anaheim,CA,92802,09/25/2019,175.0,Condominium,NaN,NaN,NaN,...,560703.0,3177315.0,3738018.0,42394 (1.13 %),3,88.0,71.0,127.0,4,2019
5314,1600 S Disneyland Dr,Anaheim,CA,92802,09/24/2019,132.0,Condominium,NaN,NaN,NaN,...,565102.0,3202243.0,3767345.0,42724 (1.13 %),3,88.0,71.0,127.0,4,2019
7038,1600 S Disneyland Dr,Anaheim,CA,92802,09/05/2019,95.0,Condominium,NaN,NaN,NaN,...,551337.0,3124245.0,3675582.0,41692 (1.13 %),3,88.0,71.0,127.0,4,2019
8169,1600 S Disneyland Dr,Anaheim,CA,92802,08/26/2019,134.0,Condominium,NaN,NaN,NaN,...,558428.0,3164423.0,3722851.0,42224 (1.13 %),3,88.0,71.0,127.0,4,2019
9458,1600 S Disneyland Dr,Anaheim,CA,92802,08/13/2019,177.0,Condominium,NaN,NaN,NaN,...,550547.0,3119765.0,3670312.0,41633 (1.13 %),3,88.0,71.0,127.0,4,2019


In [24]:
tmp_df.info_sold_price

1360      6000.0
1896     20000.0
2520     35500.0
3986     29000.0
4172     30500.0
5068     38500.0
5314     29000.0
7038     21000.0
8169     29500.0
9458     39000.0
9544     45500.0
9560     36500.0
12728    27000.0
13354    47500.0
Name: info_sold_price, dtype: float64

Đối với những dòng có cùng `address_street` mà lại khác `date_sold` và `info_sold_price` thì nên bỏ là hợp lý và khó có thể lấy được chính xác giá trị `sold_price`.

Tiếp tục, xem thử tất cả các dòng có `address_street` là `200 W Midway Dr ` (hay `duplicate_addresss_street[1]`)

In [25]:
tmp_df = df[df.address_street == duplicate_addresss_street[1]]
tmp_df

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
6842,200 W Midway Dr,Anaheim,CA,92805,09/06/2019,118100.0,MISCELLANEOUS,NaN,NaN,NaN,...,343285.0,488427.0,831712.0,9376 (1.12 %),3,88.0,71.0,127.0,4,2019
6845,200 W Midway Dr,Anaheim,CA,92805,09/06/2019,118100.0,MISCELLANEOUS,NaN,NaN,NaN,...,353884.0,578348.0,1078984.0,12209 (1.13 %),3,88.0,71.0,127.0,4,2019
6856,200 W Midway Dr,Anaheim,CA,92805,09/06/2019,118100.0,MISCELLANEOUS,NaN,NaN,NaN,...,11617.0,7141.0,18758.0,228 (1.21 %),3,88.0,71.0,127.0,4,2019
6858,200 W Midway Dr,Anaheim,CA,92805,09/06/2019,118100.0,MISCELLANEOUS,NaN,NaN,NaN,...,11217.0,27359.0,38576.0,450 (1.16 %),3,88.0,71.0,127.0,4,2019
6863,200 W Midway Dr,Anaheim,CA,92805,09/06/2019,118100.0,MISCELLANEOUS,NaN,NaN,NaN,...,16075.0,5782.0,21857.0,262 (1.20 %),3,88.0,71.0,127.0,4,2019
6896,200 W Midway Dr,Anaheim,CA,92805,09/06/2019,118100.0,MISCELLANEOUS,NaN,NaN,NaN,...,18636.0,0.0,18636.0,222 (1.19 %),3,88.0,71.0,127.0,4,2019
6910,200 W Midway Dr,Anaheim,CA,92805,09/06/2019,118100.0,MISCELLANEOUS,NaN,NaN,813.0,...,107357.0,22890.0,130247.0,1482 (1.13 %),3,88.0,71.0,127.0,4,2019


In [26]:
tmp_df.info_sold_price

6842    26000000.0
6845    26000000.0
6856    26000000.0
6858    26000000.0
6863    26000000.0
6896    26000000.0
6910    26000000.0
Name: info_sold_price, dtype: float64

Mặc dù có cùng `address_street`, cùng `date_sold` và cùng `info_sold_price` nhưng lại có các giá trị `taxes` thay đổi một cách kỳ lại + với các thông tin chính (`info_bedrooms`, `info_bathroooms`,...) bị thiếu. Phần các thông tin chính này có thể giải quyết bằng `fill miss value` (`mean`, `median`, `most`,...) nhưng phần `taxes` thì nhóm không biết xác định như thế nào cho hợp lý và gần chính xác nhất. Nên quyết định bỏ luôn các dòng dạng này.

In [27]:
sum(address_street_count[address_street_count > 1])

106

Có `106` dòng dữ liệu bị như thế, so với `15755` dòng dữ liệu hiện có thì chúng chưa chiếm đến `1%` nên nhóm quyết định bỏ đi là hợp lý.

In [28]:
df = df[~df.address_street.isin(duplicate_addresss_street)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15649 entries, 0 to 16010
Data columns (total 26 columns):
address_street        15603 non-null object
address_locality      15649 non-null object
address_region        15649 non-null object
address_code          15649 non-null int64
date_sold             15649 non-null object
mortgage              15649 non-null float64
info_type             15649 non-null object
info_bedrooms         14222 non-null float64
info_bathrooms        14232 non-null float64
info_size             14937 non-null float64
info_lot_size         10605 non-null float64
info_year_built       14461 non-null float64
info_sold_price       15649 non-null float64
info_property_id      15649 non-null int64
info_county           15649 non-null object
info_parcel_number    15649 non-null int64
taxes_land            15408 non-null float64
taxes_improvements    15408 non-null float64
taxes_total           15408 non-null float64
taxes_taxes           15408 non-null object
scho

In [29]:
# Ghi dữ liệu xuống file csv.
df.to_csv("data/Orange-County/data-2019.csv", sep='\t', index=False)

#### Tách dữ liệu thành 2 tập train và test theo tỉ lệ 80% 20%

In [30]:
# Đọc dữ liệu lên
data_df = pd.read_csv("data/Orange-County/data-2019.csv", sep='\t')
data_df.head()

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4,2019
1,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4,2019
2,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4,2019
3,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4,2019
4,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4,2019


In [31]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15649 entries, 0 to 15648
Data columns (total 26 columns):
address_street        15603 non-null object
address_locality      15649 non-null object
address_region        15649 non-null object
address_code          15649 non-null int64
date_sold             15649 non-null object
mortgage              15649 non-null float64
info_type             15649 non-null object
info_bedrooms         14222 non-null float64
info_bathrooms        14232 non-null float64
info_size             14937 non-null float64
info_lot_size         10605 non-null float64
info_year_built       14461 non-null float64
info_sold_price       15649 non-null float64
info_property_id      15649 non-null int64
info_county           15649 non-null object
info_parcel_number    15649 non-null int64
taxes_land            15408 non-null float64
taxes_improvements    15408 non-null float64
taxes_total           15408 non-null float64
taxes_taxes           15408 non-null object
scho

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
# Đổi tên
data_df = data_df.rename(columns={"info_sold_price": "sold_price"})

In [34]:
# Tách dữ liệu theo tỉ lệ 80% và 20%
train, test = train_test_split(data_df, test_size=0.2, random_state=0)
train.head().index

Int64Index([2746, 8859, 11288, 2153, 12119], dtype='int64')

&#9889; Nhóm đã cố định `random_state` trong `train_test_split` để đảm bảo kết quả của thầy ra giống với của nhóm. Kết quả của câu lệnh `train.head().index` của nhóm ra 5 giá trị là: [2746, 8859, 11288, 2153, 12119]. 

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12519 entries, 2746 to 2732
Data columns (total 26 columns):
address_street        12485 non-null object
address_locality      12519 non-null object
address_region        12519 non-null object
address_code          12519 non-null int64
date_sold             12519 non-null object
mortgage              12519 non-null float64
info_type             12519 non-null object
info_bedrooms         11363 non-null float64
info_bathrooms        11371 non-null float64
info_size             11946 non-null float64
info_lot_size         8484 non-null float64
info_year_built       11571 non-null float64
sold_price            12519 non-null float64
info_property_id      12519 non-null int64
info_county           12519 non-null object
info_parcel_number    12519 non-null int64
taxes_land            12321 non-null float64
taxes_improvements    12321 non-null float64
taxes_total           12321 non-null float64
taxes_taxes           12321 non-null object
sch

In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3130 entries, 12908 to 12030
Data columns (total 26 columns):
address_street        3118 non-null object
address_locality      3130 non-null object
address_region        3130 non-null object
address_code          3130 non-null int64
date_sold             3130 non-null object
mortgage              3130 non-null float64
info_type             3130 non-null object
info_bedrooms         2859 non-null float64
info_bathrooms        2861 non-null float64
info_size             2991 non-null float64
info_lot_size         2121 non-null float64
info_year_built       2890 non-null float64
sold_price            3130 non-null float64
info_property_id      3130 non-null int64
info_county           3130 non-null object
info_parcel_number    3130 non-null int64
taxes_land            3087 non-null float64
taxes_improvements    3087 non-null float64
taxes_total           3087 non-null float64
taxes_taxes           3087 non-null object
school               

In [37]:
# Ghi tập train
train.to_csv("data/Orange-County/train-data-2019.csv", sep='\t', index=False)

In [38]:
# Ghi tập test
test.to_csv("data/Orange-County/test-data-2019.csv", sep='\t', index=False)

# 4. Tiền xử lý dữ liệu

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data_df = pd.read_csv("data/Orange-County/train-data-2019.csv", sep='\t')
data_df.head()

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,13242 Amarillo Dr,Westminster,CA,92683,10/24/2019,3157.0,Single Family Residence,3.0,1.0,1090.0,...,508260.0,54780.0,563040.0,6969 (1.23 %),3,97.0,79.0,140.0,4,2019
1,716 S Palomino Ln,Anaheim,CA,92807,08/16/2019,2748.0,Single Family Residence,3.0,3.0,1582.0,...,371674.0,148526.0,520200.0,5527 (1.06 %),3,88.0,71.0,127.0,4,2019
2,19702 Lancewood Plz,Yorba Linda,CA,92886,07/23/2019,2966.0,Single Family Residence,4.0,2.0,1440.0,...,29459.0,43373.0,72832.0,1240 (1.70 %),3,22.0,12.0,44.0,4,2019
3,25885 Trabuco Rd Apt 59,Lake Forest,CA,92630,10/30/2019,1726.0,Condominium,2.0,2.0,1190.0,...,112879.0,93656.0,206535.0,2096 (1.01 %),3,29.0,23.0,42.0,4,2019
4,6742 Gate Hill Cir,Huntington Beach,CA,92648,07/15/2019,5860.0,Single Family Residence,4.0,3.0,2193.0,...,159764.0,148957.0,308721.0,3706 (1.20 %),3,57.0,47.0,82.0,4,2019


In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12519 entries, 0 to 12518
Data columns (total 26 columns):
address_street        12485 non-null object
address_locality      12519 non-null object
address_region        12519 non-null object
address_code          12519 non-null int64
date_sold             12519 non-null object
mortgage              12519 non-null float64
info_type             12519 non-null object
info_bedrooms         11363 non-null float64
info_bathrooms        11371 non-null float64
info_size             11946 non-null float64
info_lot_size         8484 non-null float64
info_year_built       11571 non-null float64
sold_price            12519 non-null float64
info_property_id      12519 non-null int64
info_county           12519 non-null object
info_parcel_number    12519 non-null int64
taxes_land            12321 non-null float64
taxes_improvements    12321 non-null float64
taxes_total           12321 non-null float64
taxes_taxes           12321 non-null object
schoo

Tập train có `12519` dòng dữ liệu và `26` cột.

In [4]:
data_df.describe(include="all")

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
count,12485,12519,12519,12519.000000,12519,1.251900e+04,12519,11363.000000,11371.000000,11946.000000,...,1.232100e+04,1.232100e+04,1.232100e+04,12321,12519.000000,12156.000000,12156.000000,12156.000000,12519.000000,12519.0
unique,12485,52,2,NaN,155,NaN,14,NaN,NaN,NaN,...,NaN,NaN,NaN,11959,NaN,NaN,NaN,NaN,NaN,NaN
top,19795 Keswick Ln,Irvine,CA,NaN,06/28/2019,NaN,Single Family Residence,NaN,NaN,NaN,...,NaN,NaN,NaN,10555 (1.12 %),NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1107,12517,NaN,232,NaN,8162,NaN,NaN,NaN,...,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,92552.015816,NaN,8.928697e+03,NaN,3.124263,2.602102,2426.583208,...,5.009780e+05,2.826592e+05,7.836531e+05,NaN,2.884016,54.516946,42.570171,82.203274,3.853822,2019.0
std,NaN,NaN,NaN,564.557227,NaN,7.526277e+04,NaN,1.008088,0.991297,7797.643516,...,1.434805e+06,1.751883e+06,2.832966e+06,NaN,0.578382,29.778698,28.960716,37.985382,0.750273,0.0
min,NaN,NaN,NaN,90620.000000,NaN,2.000000e+00,NaN,1.000000,1.000000,301.000000,...,4.410000e+02,0.000000e+00,0.000000e+00,NaN,0.000000,13.000000,1.000000,23.000000,0.000000,2019.0
25%,NaN,NaN,NaN,92630.000000,NaN,2.548000e+03,NaN,2.000000,2.000000,1264.000000,...,1.504430e+05,7.841300e+04,2.769060e+05,NaN,3.000000,29.000000,22.000000,48.000000,4.000000,2019.0
50%,NaN,NaN,NaN,92677.000000,NaN,3.339000e+03,NaN,3.000000,3.000000,1671.000000,...,3.111640e+05,1.312930e+05,4.712400e+05,NaN,3.000000,55.000000,39.000000,82.000000,4.000000,2019.0
75%,NaN,NaN,NaN,92804.000000,NaN,4.610000e+03,NaN,4.000000,3.000000,2377.750000,...,5.119080e+05,2.319820e+05,7.320000e+05,NaN,3.000000,71.000000,54.000000,122.000000,4.000000,2019.0


## 4.1. Tách tập dữ liệu thành 2 phần train (70%) và validation (30%)

In [5]:
# Tách X và y
y_sr = data_df["sold_price"] # sr là viết tắt của series
X_df = data_df.drop("sold_price", axis=1)

In [6]:
# Tách tập train và tập validation theo tỉ lệ 70%:30%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)
train_X_df.head().index

Int64Index([10747, 12214, 5916, 489, 12257], dtype='int64')

&#9889; Nhóm đã cố định `random_state` trong `train_test_split` để đảm bảo kết quả của thầy ra giống với của nhóm. Kết quả của câu lệnh `train_X_df.head().index` của nhóm ra 5 giá trị là: [10747, 12214, 5916, 489, 12257]. 

In [7]:
train_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8763 entries, 10747 to 2732
Data columns (total 25 columns):
address_street        8736 non-null object
address_locality      8763 non-null object
address_region        8763 non-null object
address_code          8763 non-null int64
date_sold             8763 non-null object
mortgage              8763 non-null float64
info_type             8763 non-null object
info_bedrooms         7951 non-null float64
info_bathrooms        7956 non-null float64
info_size             8361 non-null float64
info_lot_size         5931 non-null float64
info_year_built       8099 non-null float64
info_property_id      8763 non-null int64
info_county           8763 non-null object
info_parcel_number    8763 non-null int64
taxes_land            8632 non-null float64
taxes_improvements    8632 non-null float64
taxes_total           8632 non-null float64
taxes_taxes           8632 non-null object
school                8763 non-null int64
total_crime           84

`train_X_df` có `8763` dòng dữ liệu và `25` cột.

In [46]:
val_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 1540 to 4911
Data columns (total 25 columns):
address_street        3749 non-null object
address_locality      3756 non-null object
address_region        3756 non-null object
address_code          3756 non-null int64
date_sold             3756 non-null object
mortgage              3756 non-null float64
info_type             3756 non-null object
info_bedrooms         3412 non-null float64
info_bathrooms        3415 non-null float64
info_size             3585 non-null float64
info_lot_size         2553 non-null float64
info_year_built       3472 non-null float64
info_property_id      3756 non-null int64
info_county           3756 non-null object
info_parcel_number    3756 non-null int64
taxes_land            3689 non-null float64
taxes_improvements    3689 non-null float64
taxes_total           3689 non-null float64
taxes_taxes           3689 non-null object
school                3756 non-null int64
total_crime           365

`val_X_df` có `3756` dòng dữ liệu và `25` cột.

## 4.2. Quan sát và lựa chọn dữ liệu bằng kiến thức cá nhân trên tập train (`train_X_df` và `train_y_sr`)

### 4.2.1. Quan sát cột `address_street`

In [8]:
# https://stackoverflow.com/questions/55105280/how-to-extract-apartment-from-address-in-pandas
pat = r'^(?P<number>\d+)?(?P<direction>.\w(?= ))?(?P<street>.+(?=\bapt|\bunit)|.+(?=#)|.+)(?P<apt_unit>(?:\bapt|\bunit|#|).+)?'
lower_address_street = train_X_df.address_street.str.lower()
tmp = lower_address_street.str.extract(pat)
tmp

,number,direction,street,apt_unit
10747,6891,NaN,lawn haven dr,NaN
12214,422,NaN,larkspur ave,NaN
5916,183,NaN,luneta ln,NaN
489,16,NaN,kingfisher ct,NaN
12257,14391,NaN,flower st,NaN
5010,11721,NaN,pine st,NaN
2271,98,NaN,ashdale,NaN
11574,402,w,linden dr,NaN
6780,6991,NaN,san pedro cir,NaN
12148,543,NaN,avenida del verdor,NaN


In [13]:
train_X_df.iloc[10]

address_street        12836 Palm St Unit 7
address_locality              Garden Grove
address_region                          CA
address_code                         92840
date_sold                       06/10/2019
mortgage                              2112
info_type                      Condominium
info_bedrooms                            2
info_bathrooms                           3
info_size                             1329
info_lot_size                          NaN
info_year_built                       2010
info_property_id                1096666644
info_county              Orange County, CA
info_parcel_number                93017465
taxes_land                          251326
taxes_improvements                  190844
taxes_total                         442170
taxes_taxes                  5675 (1.28 %)
school                                   3
total_crime                             76
violent_crime                           57
property_crime                         122
foreclosure

In [ ]:
tmp["street"].value_counts()

In [ ]:
sum(tmp["street"].value_counts() > 1)

In [ ]:
len(tmp["street"].unique())

Theo kết quả tìm kiếm trên internet (https://cartographic.info/usa/street/show.php?p=california&t=Orange%20County) thì có `280` tên đường lớn tại `Orange County`. Vậy trong danh sách mà nhóm parse được có đến `6051` tên đường khác nhau, chắc bao gồm những con đường nhỏ. Việc dùng `street` cho huấn luyện mô hình sẽ không hiệu quả, vì khi đưa vào mô hình bằng phương pháp one-hot chắc chắn sẽ tạo ra số chiều rất lớn và với một số địa chỉ xuất hiện rất ít dễ dẫn đén overfitting.

In [ ]:
len(tmp[pd.notna(tmp["apt_unit"])])

Nhóm sẽ xóa cột `address_street` và thêm cột `apt_unit` được rút trích ở trên vào dữ liệu ở bước tiền xử lý dữ liệu, cột này chỉ có 2 giá trị (`True`/`False`)

### 4.2.2. Quan sát các cột `address_locality`, `address_region` và `address_code`

&#9889; Theo tìm hiểu của nhóm, `address_code` là tổng hợp từ `address_locality` và `address_region`

In [ ]:
unique_code = train_X_df.address_code.unique()
len(unique_code)

In [ ]:
for code in unique_code:
    tmp_df = train_X_df[train_X_df.address_code == code]
    print("Code: " + str(code))
    print("\n + Locality: " + str(tmp_df.address_locality.unique()))
    print("\n + Region: " + str(tmp_df.address_region.unique()))
    print("\n")

Theo nhóm tìm hiểu thì mã `code` (`address_code`) kia là mã bưu điện, mã này có thể dùng cho nhiều `locality` ở gần nhau, do đó ta kết quả như ở trên là hoàn toàn bình thường. Giờ nhóm quyết định bỏ 2 cột `address_locality` và `addres_region`, chỉ giữ lại cột `address_code` là đủ.

2. Xem thử cột `address_code` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_code = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.address_code)])
print("The number of missing values (column 'address_code'): " + str(num_miss_code))

Good. Cột `address_code` không có giá trị thiếu.

### 4.2.3. Quan sát cột `date_sold`

Đây là cột dạng chuỗi có giá trị rời rạc không thứ tự (categorical)

1. Xem thử cột `date_sold` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_date_sold = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.date_sold)])
print("The number of missing values (column 'date_sold'): " + str(num_miss_date_sold))

Good. Cột `date_sold` không có giá trị thiếu.

2. Rút trích thông tin từ cột `date_sold`

In [ ]:
train_X_df.date_sold.unique()

Cột `date_sold` được ghi dưới dạng `MM/DD/YYYY`. Giờ nhóm sẽ rút trích `Month` từ `date_sold`.

In [ ]:
month = train_X_df.date_sold.str[:2].astype(int)
sorted(month.unique())

Sở dĩ không có tháng `12` là do thời gian lấy dữ liệu của nhóm là đầu tháng `12` nên trang web chưa cập nhật. Về sau ở bước `Xây dựng pipline tiền xử lý dữ liệu`, nhóm sẽ tiến hành xóa cột `date_sold`, thêm cột `Month` và muốn thử để nguyên `11` tháng huấn luyện tốt hơn hay chia ra thành `4` mùa tốt hơn bằng cách đặt thêm cờ hiệu `month_to_seasion`.

### 4.2.4. Quan sát cột `mortgage`

Đây là cột dạng số (numerical) - float64

`mortgage` là tiền cho thuê tính theo tháng của căn nhà khi chưa được bán, nó cũng gần gần để suy ra `sold_price` rồi.

In [ ]:
train_X_df.mortgage.describe()

Xem thử cột `mortgage` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_mortgage = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.mortgage)])
print("The number of missing values (column 'mortgage'): " + str(num_miss_mortgage))

Good. Cột `mortgage` không có giá trị thiếu.

### 4.2.5. Quan sát cột `info_type`

Đây là cột dạng chuỗi có giá trị rời rạc không thứ tự (categorical)

1. Quan sát các giá trị xuất hiện trong cột và số lượng tương ứng

In [ ]:
train_X_df.info_type.value_counts()

Có một số giá trị xuất hiện rất ít, nhóm thử dùng thêm tham số `num_top_type` để chọn ra các `info_type` xuất hiện nhiều lần; còn các giá trị nằm ngoài `num_top_type` nhóm sẽ cho là giá trị `Others`. (như `BT03-TienXuLy_ChongOverfit`)

2. Xem thử cột `address_type` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_type = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.info_type)])
print("The number of missing values (column 'info_type'): " + str(num_miss_type))

Good. Cột `info_type` không có giá trị thiếu.

### 4.2.5. Quan sát cột `info_bedrooms` và cột `info_bathrooms`

Đây là cột dạng số (numerical) - float64

1. Xem thử cột `info_bedrooms` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_bedrooms = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_bedrooms)])
print("The number of missing values (column 'info_bedrooms'): " + str(num_miss_bedrooms))

Cột `info_bedrooms` có `812` giá trị bị thiếu.

In [ ]:
train_X_df.info_bedrooms.value_counts()

In [ ]:
train_X_df.info_bedrooms.describe()

In [ ]:
train_X_df.info_bedrooms.value_counts().sort_index().plot(kind='bar')

Do là `bedrooms` và các giá trị của nó là số nguyên, nên nhóm dùng `most` để lắp đầy các giá trị thiếu cho cột `info_bedrooms`

2. Xem thử cột `info_bathrooms` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_bathrooms = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_bathrooms)])
print("The number of missing values (column 'info_bathrooms'): " + str(num_miss_bathrooms))

Cột `info_bathrooms` có `807` giá trị bị thiếu.

In [ ]:
train_X_df.info_bathrooms.value_counts()

In [ ]:
train_X_df.info_bathrooms.describe()

In [ ]:
train_X_df.info_bathrooms.value_counts().sort_index().plot(kind='bar')

Tương tự nhóm cùng sẽ dùng `most` để lắp đầy các giá trị thiếu cho cột `info_bathrooms`

### 4.2.6. Quan sát cột `info_size` và cột `info_lot_size`

Đây là cột dạng số (numerical) - float64

1. Xem thử cột `info_size` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_size= train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_size)])
print("The number of missing values (column 'info_size'): " + str(num_miss_size))

Cột `info_size` có `402` giá trị bị thiếu.

In [ ]:
train_X_df.info_size.value_counts()

In [ ]:
train_X_df.info_size.describe()

In [ ]:
len(train_X_df.info_size.unique())

Điền các giá trị thiếu bằng `mean`.

2. Xem thử cột `info_lot_size` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_lot_size= train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_lot_size)])
print("The number of missing values (column 'info_size'): " + str(num_miss_lot_size))

Cột `info_size` có `2832` giá trị bị thiếu.

In [ ]:
train_X_df.info_lot_size.value_counts()

In [ ]:
train_X_df.info_lot_size.describe()

In [ ]:
len(train_X_df.info_lot_size.unique())

Điền các giá trị thiếu bằng `mean`.

### 4.2.7. Quan sát cột `info_year_built`

Đây là cột dạng số (numerical) - float64

Xem thử cột `info_year_built` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_year_built = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_year_built)])
print("The number of missing values (column 'info_size'): " + str(num_miss_year_built))

Cột `info_year_built` có `664` giá trị bị thiếu.

In [ ]:
train_X_df.info_year_built.value_counts()

In [ ]:
train_X_df.info_year_built.describe()

In [ ]:
len(train_X_df.info_year_built.unique())

### 4.2.8. Quan sát các cột `info_property_id`, `info_county` và `info_parcel_number`

- `info_property_id` là cột dạng số (numerical) - int64
- `info_county` là cột dạng chuỗi có giá trị rời rạc không thứ tự (categorical)
- `info_parcel_number` là cột dạng số (numerical) - int64

Bỏ đi các cột này, vì 2 cột `info_property_id` và `info_parcel_number` ứng với mỗi dòng dữ liệu là hoàn toàn khác nhau; còn cột `info_county` thì có giá trị là `Orange County` (do dữ liệu thu thập của Quận Cam).

### 4.2.9. Quan sát các cột `taxes_land`, `taxes_improvements`, `taxes_total` và `taxes_taxes`

Các cột này đều là cột dạng số (numerical) - float64

In [ ]:
num_miss_taxes = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.taxes_land)])
print("The number of missing values (column 'taxes'): " + str(num_miss_taxes))

Có `131` giá trị thiếu cho mỗi cột `taxes_land`, `taxes_improvements`, `taxes_total` và `taxes_taxes`

Search cùng `address_code` thì lắp đầy các giá trị thiếu bằng `mean` của các `taxes` trong cùng `address_code`

Giữ nguyên các cột `taxes_land`, `taxes_improvements` và bỏ cột `taxes_total`. Giải thích thêm ý nghĩa của các cột này:
- Cột `taxes_land` là tiền cải thuế cải tạo đất mà mình phải đóng cho chính phủ.
- Cột `taxes_improvements` là tiền mà mình phải trả do sử dụng các tiện ích của chính phủ (cắt cỏ, tiền rác, củng cố hàng rào,...)
- Cột `taxes_total` là tổng của 2 cột trên.

Đối với cột `taxes_taxes` thì nhóm sẽ rút trích số tiền phải trả trong cột này ra.

In [ ]:
train_X_df.taxes_taxes

In [ ]:
train_X_df.taxes_taxes.str.extract(r'(\d+)', expand=False)

### 4.2.10. Quan sát cột `school` và cột `foreclosures`

Các cột này đều là cột dạng số (numerical) - int64

Hai cột này không có giá trị thiếu, nên được giữ nguyên để huấn luyện.

In [ ]:
train_X_df.school.value_counts().sort_index().plot(kind='bar')

In [ ]:
train_X_df.foreclosures.value_counts().sort_index().plot(kind='bar')

### 4.2.11. Quan sát cột `total_crime`, `violent_crime` và `property_crime`

Các cột này đều là cột dạng số (numerical) - float64

Giải thích về ý nghĩa của các cột này. Giá trị mà các cột `total_crime`, `violent_crime` và `property_crime` biểu diễn được tính theo % so với tỉ lệ tội phạm quốc gia. Ví dụ `total_crime = 79` thì ở đây tổng số tội phạm là `79%` so với toàn nước (`100%`).

In [ ]:
num_miss_crime = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.total_crime)])
print("The number of missing values (column 'taxes'): " + str(num_miss_crime))

Có `265` giá trị thiếu cho mỗi cột `total_crime`, `violent_crime` và `property_crime`

In [ ]:
train_X_df.total_crime.value_counts()

Search cùng `address_code` thì lắp đầy các giá trị thiếu bằng `median` của các `crime` trong cùng `address_code`

### 4.2.12. Quan sát `correct input` (`sold_price` của ` train_y_sr`)

Đây là cột dạng số (numerical) - float64

In [ ]:
train_y_sr.describe()

In [ ]:
train_y_sr.sort_values().sort_index().plot(style='.')

## 4.3. Thiết kế pipeline tiền xử lý dữ liệu

1. Cột `address_street`: 
    - Ở đây có 4 thông tin được rút trích (xem  phần `4.2.1. Quan sát cột address_street` ở trên):
         + `number`: số nhà, thông tin này không cần thiết cho việc huấn luyện.
         + `direction`: hướng (đông, tây, nam, bắc) của đường, thông tin này không cần thiết cho việc huấn luyện.
         + `street`: tên đường, theo kết quả parse thì thu được đến 6051 tên đường lớn/nhỏ khác nhau. Nếu đưa tên đường này vào huấn luyện thì có 3 vấn đề: thứ nhất là nếu dùng one-hot để chuyển sang dạng số thì sẽ làm tăng số chiều dữ liệu lên rất nhiều (thêm đến trên 6000 thuộc tính), thứ hai do có khoảng 1500 tên đường chỉ xuất hiện duy nhất một lần trong `train_X_df`, còn các tên đường khác thì cũng có tần số xuất hiện khá thấp (cao nhất chỉ có 16 lần) nên nếu thêm vào huấn luyện có thể dẫn đến overfitting, thứ 3 là nếu như ở tập `validation` hay tập `test` mà có tên đường không nằm trong tập `train` thì cũng khó khăn để giải quyết. Nên nhóm quyết định không lấy thông tin `street` để huấn luyện.
         + `apt_unit`: cho biết căn nhà có thuộc chung cư nào hay không, thông tin này nhóm nghĩ là cần thiết cho việc huấn luyện vì nghĩ giá nhà riêng sẽ có phần nào khác với các căn nhà thuộc các khu chung cư. Vì vậy, sẽ thêm cột `is_apt` vào để huấn luyện, cột này chỉ có 2 giá trị `0`/`1` (nếu thuộc chung cư thì `1`, ngược lại thì `0`).
    - Tóm lại, các việc cần làm:
        + Rút trích ra cột `is_apt` (như mô tả ở trên) và thêm vào dữ liệu.
        + Xóa cột `address_street`.
        + Lắp đầy giá trị thiếu bằng KNN (n=5).
        + Đây là dữ liệu số, để nguyên vậy vào huấn luyện.
         
2. Xóa 2 cột `address_locality` và cột `address_region`, giữ lại cột `address_code` (lý do xem ở phần `4.2.2. Quan sát các cột address_locality, address_region và address_code` ở trên). Ở tập `train_X_df` thì không thấy dữ liệu thiếu, nếu thiếu dùng KNN (n=5) để lấp đầy. Đưa vào tập train huấn luyện thì xem tại link (https://towardsdatascience.com/transforming-categorical-information-into-usable-measures-in-a-machine-learning-model-e2910bbb3fc7).
3. Cột `date_sold`: rút trích `month` ra (xem ở phần `4.2.3. Quan sát cột date_sold`), xóa cột `date_sold` và `year_sol`, thêm vào cột `month_sold`. Ở tập `train_X_df` thì không thấy dữ liệu thiếu, nếu thiếu dùng KNN(n=5) để lấp đầy. Bỏ vào huấn luyện dùng one-hot, nhớ là thử để nguyên `11` tháng huấn luyện tốt hơn hay chia ra thành `4` mùa tốt hơn bằng cách đặt thêm cờ hiệu `month_to_seasion`.
4. Cột `mortgage`: để nguyên. Ở tập `train_X_df` thì không thấy dữ liệu thiếu, nếu thiếu dùng KNN (n=5) để lấp đầy. Dạng số, cứ bỏ thẳng vào huấn luyện.
5. Cột `info_type`: Có một số giá trị xuất hiện rất ít, nhóm thử dùng thêm tham số `num_top_types` để chọn ra các `info_type` xuất hiện nhiều lần; còn các giá trị nằm ngoài `num_top_types` nhóm sẽ cho là giá trị `Others`. (như `BT03-TienXuLy_ChongOverfit`). Ở tập `train_X_df` thì không thấy dữ liệu thiếu, nếu thiếu dùng KNN (n=5) để lấp đầy. Đưa vào huấn luyện thì dùng one-hot.
6. Hai cột `info_bedrooms` và `info_bathrooms`: lắp đầy giá trị thiều bằng `most`. Kiểu số, cứ bỏ thẳng vào huấn luyện.
7. Hai cột `info_size` và `info_lot_size`: dùng KNN (n=5) để lắp đầy giá trị thiếu. Kiểu số, cứ bỏ thẳng vào huấn luyện.
8. Xóa 3 cột` info_property_id`, `info_county` và `info_parcel_number` (lý do xem ở phần `4.2.9. Quan sát các cột info_property_id, info_county và info_parcel_number`).
9. Các cột `taxes_land`, `taxes_improvements`, `taxes_total` và `taxes_taxes`: đầu tiên rút trích `taxes` cho cột `taxes_taxes`, thay bằng dữ liệu tìm được (xem ở phần `4.2.10. Quan sát các cột taxes_land, taxes_improvements và taxes_taxes`), nhớ xóa cột `taxes_total`. Điền các giá trị thiếu bằng `mean`. Kiểu số, cứ bỏ thẳng vào huấn luyện.
10. Giữ nguyên cột `school` và cột `foreclosures`. Nếu có giá trị thiếu thì điền bằng `median`. Kiểu số, cứ bỏ thẳng vào huấn luyện.
11. Các `total_crime`, `violent_crime` và `property_crime`: lắp đầy giá trị thiếu bằng `median`. Kiểu số, cứ bỏ thẳng vào huấn luyện.

- Chuẩn hóa theo cách của thầy trong `BT03`.
- Dùng mô hình `Linear Regression` để huấn luyện mô mình thử xem.

In [14]:
import sklearn
sklearn.__version__

'0.22'

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

In [16]:
# Class AddressProcessor dùng để xử lý các cột address_street, address_locality và address_region.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.
class AddressProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        # Extract các thông tin (số nhà, hướng của đường, tên đường, tên chung cư) từ address_street.
        pat = r'^(?P<number>\d+)?(?P<direction>.\w(?= ))?(?P<street>.+(?=\bapt|\bunit)|.+(?=#)|.+)(?P<apt_unit>(?:\bapt|\bunit|#|).+)?'
        lower_address_street = X_df.address_street.str.lower()
        street_parsed = lower_address_street.str.extract(pat)
        
        # Xóa các cột dùng nữa.
        transformed_df = X_df.drop(["address_street", "address_locality", "address_region"], axis=1)
        
        # Thêm cột is_apt vào DataFrame (1 là thuộc chung cư, 0 là không không thuộc chung cư).
        transformed_df["is_apt"] = street_parsed["apt_unit"].notnull().astype(int)
        return transformed_df

In [17]:
# TEST AddressProcessor
ap = AddressProcessor()

ap.transform(train_X_df).is_apt.value_counts()

0    7285
1    1478
Name: is_apt, dtype: int64

In [18]:
# Class DateSoldProcessor dùng để xử lý các cột date_sold và year_sold.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.

class DateSoldProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        # Rút trích month từ date_sold.
        month = X_df.date_sold.str[:2]
        
        # Xóa các cột không dùng nữa.
        transformed_df = X_df.drop(["date_sold", "year_sold"], axis=1)
        
        # Thêm cột month vào DataFrame.
        transformed_df["month_sold"] = month
        return transformed_df

In [19]:
# TEST DateSoldProcessor
dsp = DateSoldProcessor()

dsp.transform(train_X_df).month_sold.value_counts()

7     1719
8     1707
9     1593
10    1356
6     1283
11    1068
5       12
4        9
3        7
2        5
1        4
Name: month_sold, dtype: int64

In [20]:
# Class InfoTypeProcessor dùng để xử lý cột info_type.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.

class InfoTypeProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_types=1):
        self.num_top_types = num_top_types

    def fit(self, X_df, y=None):
        info_type_col = X_df.info_type
        
        # Đếm các dạng type trong cột info_type
        self.type_counts_ = info_type_col.value_counts()
        types = list(self.type_counts_.index)
        
        # Lấy ra top_type theo tham số num_top_types được truyền vào
        self.top_types_ = types[:max(1, min(self.num_top_types, len(types)))]
        return self

    def transform(self, X_df, y=None):
        # Các cột không phải top_type được mặc định là "Others"
        type_sr = X_df.info_type.str.extract(f'^({"|".join(self.top_types_)})$', expand=False).fillna("Others")
        
        # Xóa đi cột info_type vì không dùng nữa.
        transformed_df = X_df.drop(["info_type"], axis=1)
        
        # Thêm cột type vào DataFrame.
        transformed_df["type"] = type_sr
        return transformed_df

In [21]:
# TEST InfoTypeProcessor
itp = InfoTypeProcessor(num_top_types=5)

itp.fit(train_X_df).transform(train_X_df).type.value_counts()

Single Family Residence    5709
Condominium                2399
MISCELLANEOUS               326
Multi-Family Dwellings      230
Contact Agent                70
Others                       29
Name: type, dtype: int64

In [22]:
# Class TaxesProcessor dùng để xử lý các cột taxes_total và taxes_taxes.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.

class TaxesProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        # Extract ra tiền thuế phải trả.
        taxes = X_df.taxes_taxes.str.extract(r'(\d+)', expand=False)
        
        # Xóa đi các cột không cần thiết.
        transformed_df = X_df.drop(["taxes_total", "taxes_taxes"], axis=1)
        
        # Thêm cột taxes_taxes với giá trị mới extract được vào.
        transformed_df["taxes_taxes"] = taxes
        return transformed_df

In [23]:
# TEST InfoTypeProcessor
tp = TaxesProcessor()

tp.fit(train_X_df).transform(train_X_df).taxes_taxes.value_counts()

10555     16
7259       8
5026       7
4127       6
4973       6
4875       6
1014       6
6076       6
4986       5
6540       5
5753       5
7190       5
2749       5
3634       5
3759       5
6029       5
2669       5
3606       5
3688       5
2545       5
3011       5
1152       5
1310       5
3346       4
1414       4
2987       4
1503       4
6333       4
5435       4
1392       4
          ..
11120      1
5255       1
9218       1
1500       1
468178     1
3375       1
10963      1
9402       1
8604       1
9858       1
4225       1
4651       1
26033      1
15809      1
10591      1
3985       1
1892       1
14799      1
94419      1
6388       1
8788       1
7023       1
5346       1
2865       1
8583       1
5278       1
4830       1
5898       1
11130      1
3864       1
Name: taxes_taxes, Length: 6348, dtype: int64

In [24]:
# Class ColumnDropper dùng để xóa các cột info_property_id, info_county, info_parcel_number và mortgage.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        return X_df.drop(["info_property_id", "info_county", "info_parcel_number", "mortgage"], axis=1)

In [ ]:
# TEST InfoTypeProcessor
cd = ColumnDropper()

cd.fit(train_X_df).transform(train_X_df)

In [ ]:
"""
class ColumnTransformerWithBaseColumn(BaseEstimator, TransformerMixin):
    def __init__(self, colTransformer, baseCol):
        self.colTransformer = colTransformer
        self.baseCol = baseCol

    def fit(self, X_df, y=None):
        self.train_X_df = X_df
        return self

    def transform(self, X_df, y=None):
        colTransformer.fit()
        transformed_df = X_df.copy()
        transformed_df["type"] = type_sr
        return transformed_df
"""

In [25]:
knn_cols = ["info_size", "info_lot_size"]
most_frequent_cols = ["info_bedrooms", "info_bathrooms"]
mean_cols = ["taxes_land", "taxes_improvements", "taxes_taxes"]
median_cols = ["school", "foreclosures", "total_crime", "violent_crime", "property_crime"]

type_pipeline = Pipeline([("most_frequent", SimpleImputer(strategy="most_frequent")),
                          ("onehot", OneHotEncoder())])

month_pipeline = Pipeline([("knn", KNNImputer()),
                           ("onehot", OneHotEncoder())])


column_transformer = ColumnTransformer([("knn", KNNImputer(), knn_cols),
                                        ("most_frequent", SimpleImputer(strategy="most_frequent"), most_frequent_cols),
                                        ("mean", SimpleImputer(strategy="mean"), mean_cols),
                                        ("median", SimpleImputer(strategy="median"), median_cols),
                                        ("type_col", type_pipeline, ["type"]),
                                        ("month_col", month_pipeline, ["month_sold"])])


preprocess_pipeline = Pipeline([("address", AddressProcessor()),
                                ("date_sold", DateSoldProcessor()),
                                ("info_type", InfoTypeProcessor()),
                                ("col_dropper", ColumnDropper()),
                                ("taxes", TaxesProcessor()),
                                ("column_transformer", column_transformer),
                                ('std_scaler', StandardScaler())])


preprocess_pipeline.fit_transform(train_X_df)

array([[-0.10631143,  0.33796655, -0.12028007, ...,  2.12154227,
        -0.42786688, -0.37254721],
       [-0.1172061 ,  0.26570461, -0.12028007, ..., -0.47135521,
        -0.42786688, -0.37254721],
       [-0.01744404,  0.07605337, -0.12028007, ..., -0.47135521,
        -0.42786688, -0.37254721],
       ...,
       [-0.02422137,  0.59831274,  0.92226617, ..., -0.47135521,
         2.33717551, -0.37254721],
       [-0.01079305, -0.76469479,  0.92226617, ...,  2.12154227,
        -0.42786688, -0.37254721],
       [-0.16965253, -1.48636171, -0.12028007, ..., -0.47135521,
        -0.42786688, -0.37254721]])

In [27]:
full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", LinearRegression())])

full_pipeline.fit(train_X_df, train_y_sr)
full_pipeline.score(val_X_df, val_y_sr)

0.3136408789715648

In [29]:
from sklearn.neighbors import KNeighborsRegressor

N = [1, 3, 5, 7, 9, 11]
for n in N:
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", KNeighborsRegressor(n_neighbors=n))])

    full_pipeline.fit(train_X_df, train_y_sr)
    accuracy = full_pipeline.score(val_X_df, val_y_sr)
    print("With n = " + str(n) + ": " + str( accuracy))

With n = 1: 0.3335764971438695
With n = 3: 0.5563609324653744
With n = 5: 0.49509515361314876
With n = 7: 0.47142579388172406
With n = 9: 0.40126272419346204
With n = 11: 0.39025520573855965


In [32]:
from sklearn.linear_model import ElasticNet
full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", ElasticNet())])

full_pipeline.fit(train_X_df, train_y_sr)
full_pipeline.score(val_X_df, val_y_sr)

0.15969629548983844

In [39]:
from sklearn.tree import DecisionTreeRegressor

for n in range(50):
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", DecisionTreeRegressor(random_state=n))])

    full_pipeline.fit(train_X_df, train_y_sr)
    accuracy = full_pipeline.score(val_X_df, val_y_sr)
    print("With random_state = " + str(n) + ": " + str( accuracy))

With random_state = 0: 0.5722515099857406
With random_state = 1: 0.6594282031095076
With random_state = 2: 0.6566259184556156
With random_state = 3: 0.6611581375591348
With random_state = 4: 0.44533449762352784
With random_state = 5: 0.6645405936205294
With random_state = 6: 0.6631862715926908
With random_state = 7: 0.5743835340180043
With random_state = 8: 0.6631558610963838
With random_state = 9: 0.6566312323556054
With random_state = 10: 0.6605436676700319
With random_state = 11: 0.5904711684430154
With random_state = 12: 0.6638718788860722
With random_state = 13: 0.6634272797009717
With random_state = 14: 0.4414296631461438
With random_state = 15: 0.6674070692180112
With random_state = 16: 0.44421280600900886
With random_state = 17: 0.6592520875289665
With random_state = 18: 0.6576586059846554
With random_state = 19: 0.5731811890902587
With random_state = 20: 0.3925537973092334
With random_state = 21: 0.6591860548750639
With random_state = 22: 0.65922161611623
With random_state = 2

In [40]:
from sklearn.tree import DecisionTreeRegressor

for n in range(49,101):
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", DecisionTreeRegressor(random_state=n))])

    full_pipeline.fit(train_X_df, train_y_sr)
    accuracy = full_pipeline.score(val_X_df, val_y_sr)
    print("With random_state = " + str(n) + ": " + str( accuracy))

With random_state = 49: 0.6645620471600402
With random_state = 50: 0.6607302841365534
With random_state = 51: 0.6616117198242735
With random_state = 52: 0.6623292295098332
With random_state = 53: 0.4375960012548138
With random_state = 54: 0.6604662126042536
With random_state = 55: 0.6600661805358318
With random_state = 56: 0.6666510358854035
With random_state = 57: 0.6619312588992788
With random_state = 58: 0.6658903523519497
With random_state = 59: 0.6603196101665347
With random_state = 60: 0.6612038547683874
With random_state = 61: 0.660917143664308
With random_state = 62: 0.5730046186404979
With random_state = 63: 0.6597074543493009
With random_state = 64: 0.6591170841352529
With random_state = 65: 0.6584597155787957
With random_state = 66: 0.6647048568435037
With random_state = 67: 0.6576702401148427
With random_state = 68: 0.6642028962892037
With random_state = 69: 0.6627331047569855
With random_state = 70: 0.6606658680630924
With random_state = 71: 0.5688077857922917
With random_

In [46]:
from sklearn.ensemble import GradientBoostingRegressor

for n in range(0,51):
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", GradientBoostingRegressor(random_state=n))])

    full_pipeline.fit(train_X_df, train_y_sr)
    accuracy = full_pipeline.score(val_X_df, val_y_sr)
    print("With random_state = " + str(n) + ": " + str( accuracy))

With random_state = 0: 0.5797544602776249
With random_state = 1: 0.5846184250929326
With random_state = 2: 0.5825289537170653
With random_state = 3: 0.5834804733750364
With random_state = 4: 0.5815403276312382
With random_state = 5: 0.5733282362850274
With random_state = 6: 0.570885526084569
With random_state = 7: 0.5824085448330651
With random_state = 8: 0.5715688370171383
With random_state = 9: 0.587646767797333
With random_state = 10: 0.5858990263302786
With random_state = 11: 0.5644105255465964
With random_state = 12: 0.567742922218794
With random_state = 13: 0.5802926716590306
With random_state = 14: 0.5817918171572329
With random_state = 15: 0.5758970343187998
With random_state = 16: 0.5674842487492889
With random_state = 17: 0.5733960224289912
With random_state = 18: 0.5812831756559881
With random_state = 19: 0.5806186232059181
With random_state = 20: 0.5603876334536342
With random_state = 21: 0.5645589862932554
With random_state = 22: 0.5858284786565429
With random_state = 23: 

https://www.kaggle.com/junkal/selecting-the-best-regression-model

In [47]:
from sklearn.ensemble import RandomForestRegressor
for n in range(10, 21):
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                              ("regression", RandomForestRegressor(n_estimators=n))])

    full_pipeline.fit(train_X_df, train_y_sr)
    accuracy = full_pipeline.score(val_X_df, val_y_sr)
    print("With n_trees = " + str(n) + ": " + str( accuracy))

With n_trees = 10: 0.6265766648337594
With n_trees = 11: 0.6408838110624613
With n_trees = 12: 0.7188171209522738
With n_trees = 13: 0.5735361868270339
With n_trees = 14: 0.6004390194499507
With n_trees = 15: 0.7111927854137117
With n_trees = 16: 0.6511239114584667
With n_trees = 17: 0.6590108321675201
With n_trees = 18: 0.6265999280208535
With n_trees = 19: 0.6860127111824526
With n_trees = 20: 0.6894075130444347


In [48]:
from sklearn.ensemble import RandomForestRegressor
for n in range(21, 31):
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                              ("regression", RandomForestRegressor(n_estimators=n))])

    full_pipeline.fit(train_X_df, train_y_sr)
    accuracy = full_pipeline.score(val_X_df, val_y_sr)
    print("With n_trees = " + str(n) + ": " + str( accuracy))

With n_trees = 21: 0.6331044052342187
With n_trees = 22: 0.6494580868361395
With n_trees = 23: 0.6251208728267221
With n_trees = 24: 0.5205448275957072
With n_trees = 25: 0.6245912878759541
With n_trees = 26: 0.6121324606140435
With n_trees = 27: 0.5739794610849208
With n_trees = 28: 0.6370140169170997
With n_trees = 29: 0.5872069711750846
With n_trees = 30: 0.6606806354034005
